In [1]:
# 파이썬≥3.5 필수
import sys
assert sys.version_info >= (3, 5)
# 공통 모듈 임포트
import numpy as np
import pandas as pd
import os
# 깔끔한 그래프 출력을 위해 %matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# 그림을 저장할 위치
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "classification"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)
def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("그림 저장:", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)
from matplotlib import font_manager, rc
import platform
path = "c:/Windows/Fonts/malgun.ttf"
if platform.system() == 'Darwin':
    rc('font', family='AppleGothic')
elif platform.system() == 'Windows':
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)

mpl.rcParams['axes.unicode_minus'] = False
# Jupyter Notebook의 출력을 소수점 이하 3자리로 제한
%precision 3
# 그래픽 출력을 조금 더 고급화하기 위한 라이브러리
import seaborn as sns

#과학 기술 통계 라이브러리
import scipy as sp
from scipy import stats
# 사이킷런 ≥0.20 필수 : 0.20에서 데이터 변환을 위한 Transformer클래스가 추가됨
import sklearn
assert sklearn.__version__ >= "0.20"

# 데이터를 분할할 때 동일한 분할을 만들기 위해서
# 모델을 만드는 작업을 여러 번에 걸쳐서 하는 경우 시드가 변경이 되서 훈련용 데이터가
# 자주 변경이 되면 결국 모든 데이터를 가지고 모델을 생성하는 결과
# Overfit이 될 가능성이 높아짐
np.random.seed(42)

import pandas as pd
import numpy as np


### 뉴스 그룹 분류

In [2]:
# 데이터 가져오기
from sklearn.datasets import fetch_20newsgroups

news_data = fetch_20newsgroups(subset='all', random_state=42)
# sklearn에서는 datasets 서브 패키지를 이용해서 가져온 데이터는 dict
# 가져온 데이터의 key를 확인
print(news_data.keys())

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])


In [6]:
print(news_data.filenames)
print(news_data.DESCR) #데이터에 대한 설명

['C:\\Users\\USER\\scikit_learn_data\\20news_home\\20news-bydate-test\\rec.sport.hockey\\54367'
 'C:\\Users\\USER\\scikit_learn_data\\20news_home\\20news-bydate-train\\comp.sys.ibm.pc.hardware\\60215'
 'C:\\Users\\USER\\scikit_learn_data\\20news_home\\20news-bydate-train\\talk.politics.mideast\\76120'
 ...
 'C:\\Users\\USER\\scikit_learn_data\\20news_home\\20news-bydate-train\\comp.sys.ibm.pc.hardware\\60695'
 'C:\\Users\\USER\\scikit_learn_data\\20news_home\\20news-bydate-train\\comp.graphics\\38319'
 'C:\\Users\\USER\\scikit_learn_data\\20news_home\\20news-bydate-test\\rec.autos\\103195']
.. _20newsgroups_dataset:

The 20 newsgroups text dataset
------------------------------

The 20 newsgroups dataset comprises around 18000 newsgroups posts on
20 topics split in two subsets: one for training (or development)
and the other one for testing (or for performance evaluation). The split
between the train and test set is based upon a messages posted before
and after a specific date.

This m

In [3]:
# 분포 확인 - 분포가 한 쪽으로 치우치게 되면 데이터를 층화 추출을 할 것인지 아니면 오버 샘플링이나 언더 샘플링인지 또는 로그 변환을 할 것인지 선택
print(pd.Series(news_data.target).value_counts().sort_index())
# 타겟은 숫자로 되어 있음을 확인함


0     799
1     973
2     985
3     982
4     963
5     988
6     975
7     990
8     996
9     994
10    999
11    991
12    984
13    990
14    987
15    997
16    910
17    940
18    775
19    628
dtype: int64


In [4]:
# target의 클래스 이름확인
print(news_data.target_names)
# 데이터 확인
print(news_data.data[0])

['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']
From: Mamatha Devineni Ratnam <mr47+@andrew.cmu.edu>
Subject: Pens fans reactions
Organization: Post Office, Carnegie Mellon, Pittsburgh, PA
Lines: 12
NNTP-Posting-Host: po4.andrew.cmu.edu



I am sure some bashers of Pens fans are pretty confused about the lack
of any kind of posts about the recent Pens massacre of the Devils. Actually,
I am  bit puzzled too and a bit relieved. However, I am going to put an end
to non-PIttsburghers' relief with a bit of praise for the Pens. Man, they
are killing those Devils worse than I thought. Jagr just showed you why
he is much better than his regular season st

In [9]:
# 데이터 가져오기
# headers 나 footers, quotes를 제거하고 가져오기
train_news = fetch_20newsgroups(subset='train', remove=('headers','footers','quotes'),
                                random_state=42)

# 훈련 데이터 생성
X_train = train_news.data
y_train = train_news.target # 실제 데이터에서 이게 없으면 군집
print(type(X_train))

test_news = fetch_20newsgroups(subset='test', remove=('headers','footers','quotes'),
                                random_state=42)

X_test = test_news.data
y_test = test_news.target # 실제 데이터에서 이게 없으면 군집
print(type(X_test))



<class 'list'>
<class 'list'>


#### 피처 벡터화 - 문자열을 벡터화

In [12]:
from sklearn.feature_extraction.text import CountVectorizer

# Count Vectorization으로 feature extraction 변환 수행.
# 단어가 등장한 개수 기반의 벡터화를 위한 인스턴스 생성
cnt_vect = CountVectorizer()

# 벡터화
cnt_vect.fit(X_train)
X_train_cnt_vect = cnt_vect.transform(X_train)
# print(type(X_train_cnt_vect)) # 타입은 희소 행렬

# 학습 데이터로 fit( )된 CountVectorizer를 이용하여 테스트 데이터를 feature extraction 변환 수행. 
X_test_cnt_vect = cnt_vect.transform(X_test)

print('학습 데이터 Text의 CountVectorizer Shape:',X_train_cnt_vect.shape)

학습 데이터 Text의 CountVectorizer Shape: (11314, 101631)


#### 로지스틱 회귀를 이용한 분류

In [15]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# LogisticRegression을 이용하여 학습/예측/평가 수행. 
lr_clf = LogisticRegression(solver="lbfgs", max_iter=1000)
lr_clf.fit(X_train_cnt_vect , y_train)
pred = lr_clf.predict(X_test_cnt_vect)
print('CountVectorized Logistic Regression 의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test,pred)))

CountVectorized Logistic Regression 의 예측 정확도는 0.597


#### TF-IDF 이용

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer()
tfidf_vect.fit(X_train)
X_train_tfidf_vect = tfidf_vect.transform(X_train)
X_test_tfidf_vect = tfidf_vect.transform(X_test)

lr_clf = LogisticRegression(solver="lbfgs", max_iter=1000)
lr_clf.fit(X_train_tfidf_vect , y_train)
pred = lr_clf.predict(X_test_tfidf_vect)
print('TF-IDF Logistic Regression 의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test ,pred)))

TF-IDF Vectorized Logistic Regression 의 예측 정확도는 0.674


In [19]:
# TF-IDF에 파라미터를 설정하기(stop_words='english', ngram_range=(1,2), max_df=300)

from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2), max_df=300 )
tfidf_vect.fit(X_train)
X_train_tfidf_vect = tfidf_vect.transform(X_train)
X_test_tfidf_vect = tfidf_vect.transform(X_test)

lr_clf = LogisticRegression(solver="lbfgs", max_iter=1000)
lr_clf.fit(X_train_tfidf_vect , y_train)
pred = lr_clf.predict(X_test_tfidf_vect)
print('TF-IDF Vectorized Logistic Regression 의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test ,pred)))

TF-IDF Vectorized Logistic Regression 의 예측 정확도는 0.692


## 감성 분석

### 나이브 베이즈를 이용한 감성 분석

In [20]:
### 훈련 데이터 만들기
train = [('i like you', 'pos'), 
         ('i do not like you', 'neg'),
         ('i hate you', 'neg'), 
         ('i do not hate you', 'pos'),
        ('i love you', 'pos'),
        ('I do not love you', 'neg')]

# 등장한 모든 단어 찾기
from nltk.tokenize import word_tokenize
import nltk

# 단어 단위로 분할해서 등장한 단어 전부 추출
all_words = set(word.lower() for sentence in train for word in word_tokenize(sentence[0]))
print(all_words)



{'like', 'not', 'do', 'i', 'hate', 'you', 'love'}


In [22]:
# 분류기 만들기

# 단어 토큰화 - 각 문장에 단어의 포함 여부를 만들고 감성을 기록
# 위에서 등장한 단어를 추출한 것으로(총7개 단어) True, False로 포함여부를 판단
t = [({word: (word in word_tokenize(x[0])) for word in all_words}, x[1])
                                                        for x in train]
print(t)



[({'like': True, 'not': False, 'do': False, 'i': True, 'hate': False, 'you': True, 'love': False}, 'pos'), ({'like': True, 'not': True, 'do': True, 'i': True, 'hate': False, 'you': True, 'love': False}, 'neg'), ({'like': False, 'not': False, 'do': False, 'i': True, 'hate': True, 'you': True, 'love': False}, 'neg'), ({'like': False, 'not': True, 'do': True, 'i': True, 'hate': True, 'you': True, 'love': False}, 'pos'), ({'like': False, 'not': False, 'do': False, 'i': True, 'hate': False, 'you': True, 'love': True}, 'pos'), ({'like': False, 'not': True, 'do': True, 'i': False, 'hate': False, 'you': True, 'love': True}, 'neg')]


In [23]:
# 텍스트 분류를 위한 나이브베이즈 분류기를 이용해서 모델을 생성
classifier = nltk.NaiveBayesClassifier.train(t)
classifier.show_most_informative_features()

Most Informative Features
                      do = False             pos : neg    =      1.7 : 1.0
                      do = True              neg : pos    =      1.7 : 1.0
                     not = False             pos : neg    =      1.7 : 1.0
                     not = True              neg : pos    =      1.7 : 1.0
                       i = True              pos : neg    =      1.4 : 1.0
                    hate = False             neg : pos    =      1.0 : 1.0
                    hate = True              neg : pos    =      1.0 : 1.0
                    like = False             neg : pos    =      1.0 : 1.0
                    like = True              neg : pos    =      1.0 : 1.0


In [24]:
# 예측
# 샘플 문장 테스트 
test_sentence = 'i do not like jessica'
test_sent_features = {word.lower():(word in word_tokenize(test_sentence.lower()))
                     for word in all_words}
print(test_sent_features)
print(classifier.classify(test_sent_features))

{'like': True, 'not': True, 'do': True, 'i': True, 'hate': False, 'you': False, 'love': False}
neg


### 한글 감성 분석

In [25]:
train = [('나는 당신을 사랑합니다', 'pos'), 
         ('나는 당신을 사랑하지 않아요', 'neg'),
         ('나는 당신을 만나는 것이 지루합니다', 'neg'),
         ('나는 당신을 만나는 것이 지루하지 않습니다', 'pos'),
         ('나는 당신이 좋습니다', 'pos'),
         ('나는 당신이 좋지 않습니다', 'neg'),
         ('나는 당신과 노는 것이 즐겁습니다', 'pos'),
         ('나는 당신과 노는 것이 즐겁지 않습니다', 'neg'),
        ('나는 제시카와 함께 있는 것이 즐겁습니다', 'pos'),
        ('나는 일을 하는 것이 즐겁지 않습니다', 'neg'),
         ('나는 일이 너무 힘들어', 'neg')]

#단순 단어 분류 – 조사가 다른 경우 다른 단어로 구분됨'
#영어가 있을 수도 있어서 lower 써주기
#단어 단위로 분할해서 등장한 단어 전부 추출
all_words = set(word.lower() for sentence in train
                        for word in word_tokenize(sentence[0]))
print(all_words)

{'함께', '즐겁습니다', '사랑하지', '당신이', '힘들어', '않습니다', '너무', '당신을', '나는', '좋지', '노는', '지루하지', '않아요', '하는', '제시카와', '즐겁지', '있는', '일을', '사랑합니다', '당신과', '일이', '좋습니다', '만나는', '것이', '지루합니다'}


In [27]:
from konlpy.tag import Twitter
twitter = Twitter()

#문장 단위로 형태소 분석기에 넣어서 단어와 품사를 /로 구분해서 추출해주는 함수
def tokenize(doc):
    return ["/".join(t) for t in twitter.pos(doc, norm=True, stem=True)]

train_docs = [(tokenize(row[0]), row[1]) for row in train]

print(train_docs)

[(['나/Noun', '는/Josa', '당신/Noun', '을/Josa', '사랑/Noun', '하다/Verb'], 'pos'), (['나/Noun', '는/Josa', '당신/Noun', '을/Josa', '사랑/Noun', '하다/Verb', '않다/Verb'], 'neg'), (['나/Noun', '는/Josa', '당신/Noun', '을/Josa', '만나다/Verb', '것/Noun', '이/Josa', '지루하다/Adjective'], 'neg'), (['나/Noun', '는/Josa', '당신/Noun', '을/Josa', '만나다/Verb', '것/Noun', '이/Josa', '지루하다/Adjective', '않다/Verb'], 'pos'), (['나/Noun', '는/Josa', '당신/Noun', '이/Josa', '좋다/Adjective'], 'pos'), (['나/Noun', '는/Josa', '당신/Noun', '이/Josa', '좋다/Adjective', '않다/Verb'], 'neg'), (['나/Noun', '는/Josa', '당신/Noun', '과/Josa', '노/Noun', '는/Josa', '것/Noun', '이/Josa', '즐겁다/Adjective'], 'pos'), (['나/Noun', '는/Josa', '당신/Noun', '과/Josa', '노/Noun', '는/Josa', '것/Noun', '이/Josa', '즐겁다/Adjective', '않다/Verb'], 'neg'), (['나/Noun', '는/Josa', '제시카/Noun', '와/Josa', '함께/Adverb', '있다/Adjective', '것/Noun', '이/Josa', '즐겁다/Adjective'], 'pos'), (['나/Noun', '는/Josa', '일/Noun', '을/Josa', '하다/Verb', '것/Noun', '이/Josa', '즐겁다/Adjective', '않다/Verb'], 'neg'), (['나/Noun', '는/Josa'

c:\Users\USER\anaconda3\lib\site-packages\konlpy\tag\_okt.py:17: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [28]:
# 단어만 추출하기(pos, neg로 분류된 부분은 빼고 단어만 추출)
# 형태소 분류를 한 후 단어 분류
tokens = [t for d in train_docs for t in d[0]]
print(tokens)

['나/Noun', '는/Josa', '당신/Noun', '을/Josa', '사랑/Noun', '하다/Verb', '나/Noun', '는/Josa', '당신/Noun', '을/Josa', '사랑/Noun', '하다/Verb', '않다/Verb', '나/Noun', '는/Josa', '당신/Noun', '을/Josa', '만나다/Verb', '것/Noun', '이/Josa', '지루하다/Adjective', '나/Noun', '는/Josa', '당신/Noun', '을/Josa', '만나다/Verb', '것/Noun', '이/Josa', '지루하다/Adjective', '않다/Verb', '나/Noun', '는/Josa', '당신/Noun', '이/Josa', '좋다/Adjective', '나/Noun', '는/Josa', '당신/Noun', '이/Josa', '좋다/Adjective', '않다/Verb', '나/Noun', '는/Josa', '당신/Noun', '과/Josa', '노/Noun', '는/Josa', '것/Noun', '이/Josa', '즐겁다/Adjective', '나/Noun', '는/Josa', '당신/Noun', '과/Josa', '노/Noun', '는/Josa', '것/Noun', '이/Josa', '즐겁다/Adjective', '않다/Verb', '나/Noun', '는/Josa', '제시카/Noun', '와/Josa', '함께/Adverb', '있다/Adjective', '것/Noun', '이/Josa', '즐겁다/Adjective', '나/Noun', '는/Josa', '일/Noun', '을/Josa', '하다/Verb', '것/Noun', '이/Josa', '즐겁다/Adjective', '않다/Verb', '나/Noun', '는/Josa', '일이/Modifier', '너무/Adverb', '힘들다/Adjective']


In [29]:
# 분류기 만들기 - 문장에 단어의 존재 여부를 확인해주는 함수
def term_exists(doc):
    return {word:(word in set(doc)) for word in tokens}
# 모든 문장을 해석해서 단어의 존재 여부와 감성을 가진 튜플의 list를 생성
train_xy = [(term_exists(d), c) for d, c in train_docs]
print(train_xy)

[({'나/Noun': True, '는/Josa': True, '당신/Noun': True, '을/Josa': True, '사랑/Noun': True, '하다/Verb': True, '않다/Verb': False, '만나다/Verb': False, '것/Noun': False, '이/Josa': False, '지루하다/Adjective': False, '좋다/Adjective': False, '과/Josa': False, '노/Noun': False, '즐겁다/Adjective': False, '제시카/Noun': False, '와/Josa': False, '함께/Adverb': False, '있다/Adjective': False, '일/Noun': False, '일이/Modifier': False, '너무/Adverb': False, '힘들다/Adjective': False}, 'pos'), ({'나/Noun': True, '는/Josa': True, '당신/Noun': True, '을/Josa': True, '사랑/Noun': True, '하다/Verb': True, '않다/Verb': True, '만나다/Verb': False, '것/Noun': False, '이/Josa': False, '지루하다/Adjective': False, '좋다/Adjective': False, '과/Josa': False, '노/Noun': False, '즐겁다/Adjective': False, '제시카/Noun': False, '와/Josa': False, '함께/Adverb': False, '있다/Adjective': False, '일/Noun': False, '일이/Modifier': False, '너무/Adverb': False, '힘들다/Adjective': False}, 'neg'), ({'나/Noun': True, '는/Josa': True, '당신/Noun': True, '을/Josa': True, '사랑/Noun': False, '하다/Verb': False,

In [30]:
classifier = nltk.NaiveBayesClassifier.train(train_xy)
classifier.show_most_informative_features()

Most Informative Features
                 않다/Verb = True              neg : pos    =      2.6 : 1.0
                 않다/Verb = False             pos : neg    =      2.1 : 1.0
                 당신/Noun = False             neg : pos    =      1.4 : 1.0
                  이/Josa = False             neg : pos    =      1.4 : 1.0
                 하다/Verb = True              neg : pos    =      1.4 : 1.0
                  와/Josa = False             neg : pos    =      1.2 : 1.0
            있다/Adjective = False             neg : pos    =      1.2 : 1.0
                제시카/Noun = False             neg : pos    =      1.2 : 1.0
               함께/Adverb = False             neg : pos    =      1.2 : 1.0
                  것/Noun = False             neg : pos    =      1.2 : 1.0


In [31]:
### 샘플 문장 테스트
test_sentence = [("여섯시 부터 일을 해야해")]
test_docs = twitter.pos(test_sentence[0])
print(test_docs)
test_sent_features = {word: (word in tokens) for word in test_docs}
print(test_sent_features)
print(classifier.classify(test_sent_features))

[('여섯시', 'Noun'), ('부터', 'Noun'), ('일', 'Noun'), ('을', 'Josa'), ('해야해', 'Verb')]
{('여섯시', 'Noun'): False, ('부터', 'Noun'): False, ('일', 'Noun'): False, ('을', 'Josa'): False, ('해야해', 'Verb'): False}
neg


## IMDB 영화평 데이터를 이용한 지도 학습 기반 감성 분석

In [32]:
review_df = pd.read_csv('./data/data/data/imdb/labeledTrainData.tsv', header=0, sep="\t", quoting=3)
review_df.head(3)
# id : review를 구분하기 위한 데이터
# sentiment : 감성(1이면 긍정 0이면 부정)
# review가 review 데이터

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."


In [33]:
# 정규식을 이용해서 불필요한 데이터 제거
# 정규식 모듈
import re

# <br> html 태그는 replace 함수로 공백으로 변환
review_df['review'] = review_df['review'].str.replace('<br />',' ')

# 파이썬의 정규 표현식 모듈인 re를 이용하여 영어 문자열이 아닌 문자는 모두 공백으로 변환 
review_df['review'] = review_df['review'].apply( lambda x : re.sub("[^a-zA-Z]", " ", x))
print(review_df.head())

         id  sentiment                                             review
0  "5814_8"          1   With all this stuff going down at the moment ...
1  "2381_9"          1     The Classic War of the Worlds   by Timothy ...
2  "7759_3"          0   The film starts with a manager  Nicholas Bell...
3  "3630_4"          0   It must be assumed that those who praised thi...
4  "9495_8"          1   Superbly trashy and wondrously unpretentious ...


In [37]:
# 훈련 데이터와 테스트 데이터 분할
from sklearn.model_selection import train_test_split

class_df = review_df['sentiment']
feature_df = review_df.drop(['id','sentiment'], axis=1, inplace=False)

#print(class_df)
#print(feature_df)

X_train, X_test, y_train, y_test= train_test_split(feature_df, class_df, test_size=0.3, random_state=42)
X_train.shape, X_test.shape

((17500, 1), (7500, 1))

In [38]:
# 훈련 및 예측
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score

# 피처를 벡터화 시킨 후 로지스틱 회귀 진행 - 파이프라인으로 한번에 진행
# 스톱 워드는 English, filtering, ngram은 (1,2)로 설정해 CountVectorization수행. 
# LogisticRegression의 C는 10으로 설정. 
pipeline = Pipeline([
    ('cnt_vect', CountVectorizer(stop_words='english', ngram_range=(1,2) )),
    ('lr_clf', LogisticRegression(C=10))])

# Pipeline 객체를 이용하여 fit(), predict()로 학습/예측 수행. predict_proba()는 roc_auc때문에 수행.  
pipeline.fit(X_train['review'], y_train)
pred = pipeline.predict(X_test['review'])
pred_probs = pipeline.predict_proba(X_test['review'])[:,1]

print('예측 정확도는 {0:.4f}, ROC-AUC는 {1:.4f}'.format(accuracy_score(y_test ,pred),
                                         roc_auc_score(y_test, pred_probs)))

c:\Users\USER\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


예측 정확도는 0.8847, ROC-AUC는 0.9508


In [39]:
# 벡터화를 수정(CountVectorizer -> TfidfVectorizer)
pipeline = Pipeline([
    ('cnt_vect', TfidfVectorizer(stop_words='english', ngram_range=(1,2) )),
    ('lr_clf', LogisticRegression(C=10))])


pipeline.fit(X_train['review'], y_train)
pred = pipeline.predict(X_test['review'])
pred_probs = pipeline.predict_proba(X_test['review'])[:,1]

print('예측 정확도는 {0:.4f}, ROC-AUC는 {1:.4f}'.format(accuracy_score(y_test ,pred),
                                         roc_auc_score(y_test, pred_probs)))

예측 정확도는 0.8916, ROC-AUC는 0.9592


### 네이버 식당 리뷰 데이터를 이용한 한글 지도 학습 기반의 감성 분석

#### 데이터 가져오기

In [40]:
df = pd.read_csv("./data/data/data/review_data.csv")
print(df.head())

   score                      review  y
0      5            친절하시고 깔끔하고 좋았습니다  1
1      5                  조용하고 고기도 굿  1
2      4      갈비탕과 냉면, 육회비빔밥이 맛있습니다.  1
3      4  대체적으로 만족하나\n와인의 구성이 살짝 아쉬움  1
4      5       고기도 맛있고 서비스는 더 최고입니다~  1


In [41]:
import re

# 텍스트 정제 함수 : 한글 이외의 문자는 전부 제거
# 한글만 추출해주는 함수
def text_cleaning(text):
    # 한글의 정규표현식으로 한글만 추출
    hangul = re.compile('[^ ㄱ-ㅣ 가-힣]') # ㄱ-ㅣ 를 지정하지 않으면 ㅋㅋㅋ 같은 것들은 추출되지 않음
    result = hangul.sub('', text)
    return result

# 정규표현식을 이용해서 한글만 추출한 데이터는 ko_text로 저장하고 review 칼럼은 삭제
df['ko_text'] = df['review'].apply(lambda x: text_cleaning(x))
del df['review']
df.head()


,score,y,ko_text
0,5,1,친절하시고 깔끔하고 좋았습니다
1,5,1,조용하고 고기도 굿
2,4,1,갈비탕과 냉면 육회비빔밥이 맛있습니다
3,4,1,대체적으로 만족하나와인의 구성이 살짝 아쉬움
4,5,1,고기도 맛있고 서비스는 더 최고입니다


### 형태소 분석

In [42]:
from konlpy.tag import Okt

# konlpy라이브러리로 텍스트 데이터에서 형태소를 추출합니다.
def get_pos(x):
    tagger = Okt()
    pos = tagger.pos(x)
    pos = ['{}/{}'.format(word,tag) for word, tag in pos]
    return pos

# 하나의 데이터로 형태소 추출 동작을 테스트
result = get_pos(df['ko_text'][0])
print(result)

['친절하시고/Adjective', '깔끔하고/Adjective', '좋았습니다/Adjective']


#### 피처 벡터화

In [43]:
from sklearn.feature_extraction.text import CountVectorizer

# 형태소를 벡터 형태의 학습 데이터셋(X 데이터)으로 변환
index_vectorizer = CountVectorizer(tokenizer = lambda x: get_pos(x))
X = index_vectorizer.fit_transform(df['ko_text'].tolist())

X.shape

c:\Users\USER\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


(545, 3030)

#### 피처 확인

In [45]:
# 피처 확인
print(str(index_vectorizer.vocabulary_)[:100])
print(df['ko_text'][0])
print(X[0])

{'친절하시고/Adjective': 2647, '깔끔하고/Adjective': 428, '좋았습니다/Adjective': 2403, '조용하고/Adjective': 2356, '고
친절하시고 깔끔하고 좋았습니다
  (0, 2647)	1
  (0, 428)	1
  (0, 2403)	1


In [48]:
# tfidf 벡터화 진행
from sklearn.feature_extraction.text import TfidfTransformer

# 형태소를 벡터 형태의 학습 데이터셋(X 데이터)으로 변환
tfidf_vectorizer = TfidfTransformer()
X = tfidf_vectorizer.fit_transform(X)

print(X.shape)
print(X[0]) # 가중치 확인 가능

(545, 3030)
  (0, 428)	0.7573091319965198
  (0, 2403)	0.4011484274975501
  (0, 2647)	0.5153278739898709


#### 학습용 데이터와 훈련용 데이터를 생성

In [50]:
from sklearn.model_selection import train_test_split

y = df['y']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)
print(X_train.shape)
print(X_test.shape)

(381, 3030)
(164, 3030)


#### 모델 훈련 및 평가

In [51]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# 로지스틱 회귀모델을 학습합니다.
lr = LogisticRegression(random_state=42)
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
y_pred_probability = lr.predict_proba(X_test)[:,1]

# 로지스틱 회귀모델의 성능을 평가합니다.
print("accuracy: %.2f" % accuracy_score(y_test, y_pred))
print("Precision : %.3f" % precision_score(y_test, y_pred))
print("Recall : %.3f" % recall_score(y_test, y_pred))
print("F1 : %.3f" % f1_score(y_test, y_pred))

accuracy: 0.90
Precision : 0.896
Recall : 1.000
F1 : 0.945


In [52]:
# 오차행렬
from sklearn.metrics import confusion_matrix

# Confusion Matrix를 출력합니다.
confmat = confusion_matrix(y_true=y_test, y_pred=y_pred)
# 전부 1로 예측
print(confmat)

[[  0  17]
 [  0 147]]


#### 타겟 분포 확인

In [53]:
df['y'].value_counts()

1    492
0     53
Name: y, dtype: int64

#### UnderSampling

In [57]:
# 0인 데이터가 53개가 있으니 1,0을 각각 50개 정도로 샘플링 하기(같은 비율로 추출)
# 비율이 높은 데이터(y가 1인 데이터)의 비율을 낮추기(UnderSampling)
positive_random_idx = df[df['y']==1].sample(50, random_state=30).index.tolist()
negative_random_idx = df[df['y']==0].sample(50, random_state=30).index.tolist()

# 랜덤 데이터로 데이터셋을 나누기
random_idx = positive_random_idx + negative_random_idx
sample_X = X[random_idx, :]
y = df['y'][random_idx]
X_train, X_test, y_train, y_test = train_test_split(sample_X, y, test_size=0.1)
print(X_train.shape)
print(X_test.shape)

(90, 3030)
(10, 3030)


In [58]:
# 평가
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# 로지스틱 회귀모델을 학습합니다.
lr = LogisticRegression(random_state=42)
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
y_pred_probability = lr.predict_proba(X_test)[:,1]

# 로지스틱 회귀모델의 성능을 평가합니다.
print("accuracy: %.2f" % accuracy_score(y_test, y_pred))
print("Precision : %.3f" % precision_score(y_test, y_pred))
print("Recall : %.3f" % recall_score(y_test, y_pred))
print("F1 : %.3f" % f1_score(y_test, y_pred))

accuracy: 0.80
Precision : 0.800
Recall : 0.800
F1 : 0.800


In [59]:
# 오차행렬
from sklearn.metrics import confusion_matrix

# Confusion Matrix를 출력합니다.
confmat = confusion_matrix(y_true=y_test, y_pred=y_pred)

print(confmat)

[[4 1]
 [1 4]]


## 토픽 모델링

In [69]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.decomposition import LatentDirichletAllocation # 차원 축소와 관련(기억해두기!)

# 데이터를 가져올 카테고리 설정
cats = ['rec.motorcycles', 'rec.sport.baseball', 'comp.graphics', 'comp.windows.x',
        'talk.politics.mideast', 'soc.religion.christian', 'sci.electronics', 'sci.med']
# 카테고리에 해당하는 데이터만 가져오기
news_df= fetch_20newsgroups(subset='all',remove=('headers', 'footers', 'quotes'), 
                            categories=cats, random_state=42)

In [61]:
count_vect = CountVectorizer(max_df=0.95, max_features=1000, min_df=2, stop_words='english', ngram_range=(1,2))
feat_vect = count_vect.fit_transform(news_df.data)
print('CountVectorizer Shape:', feat_vect.shape)

CountVectorizer Shape: (7862, 1000)


In [67]:
# 토픽 모델링
# 토픽의 개수는 8개
lda = LatentDirichletAllocation(n_components=8, random_state=0)
lda.fit(feat_vect)
print(lda.components_.shape)

(8, 1000)


In [71]:
# 각 토픽에서 중요한 10개 단어 추출

def display_topics(model, feature_names, no_top_words):
    for topic_index, topic in enumerate(model.components_):
        print('Topic #',topic_index)

        # components_ array에서 가장 값이 큰 순으로 정렬했을 때, 그 값의 array index를 반환. 
        topic_word_indexes = topic.argsort()[::-1]
        top_indexes=topic_word_indexes[:no_top_words]
        
        # top_indexes대상인 index별로 feature_names에 해당하는 word feature 추출 후 join으로 concat
        # 단어 찾아주는 작업
        feature_concat = ' '.join([feature_names[i] for i in top_indexes])                
        print(feature_concat)

# sklearn의 최신 버전에서 get_feature_names 함수가 없어지고 count_vect.get_feature_names_out로 변경
# CountVectorizer객체내의 전체 word들의 명칭을 get_features_names( )를 통해 추출
feature_names = count_vect.get_feature_names_out()

# Topic별 가장 연관도가 높은 word를 15개만 추출
display_topics(lda, feature_names, 15)

Topic # 0
year 10 game medical health team 12 20 disease 1993 cancer games years patients 92
Topic # 1
don just know like said people time think didn ve right going say ll did
Topic # 2
image file jpeg program gif output format images color files entry 00 use bit 03
Topic # 3
like think don just use does know good time people used question read point make
Topic # 4
armenian israel jews armenians turkish people israeli jewish government war dos dos turkey arab 000 armenia
Topic # 5
edu com available graphics ftp window motif data pub mail widget information use server mit
Topic # 6
god jesus people church christ believe christian does christians say bible faith sin think paul
Topic # 7
thanks dos use windows using does help like need display know software server pc problem


## 텍스트 군집

In [4]:
# ./data/data/data/OpinosisDataset1.0/topics 디렉토리 안의 파일의 내용을 전부 읽기

import glob, os
import platform

# path = "" 대신에 path = None 으로 써도 가능(문자열이기 때문에 가능)
path = ""
if platform.system() == 'Windows':
    path = "data\data\data\OpinosisDataset1.0\topics"
else : 
    path = "data/data/data/OpinosisDataset1.0/topics"

# 디렉토리 이름을 생성
#path = "./data/data/data/OpinosisDataset1.0/topics"
#path = "data\data\data\OpinosisDataset1.0\topics"
# 디렉토리 안의 모든 파일이름을 list로 생성
all_files = glob.glob(os.path.join(path, "*.data"))
# print(all_files)

# 파일 이름을 저장할 list
filename_list = []
# 파일 내용을 저장할 list
opinion_text = []

# 파일 경로를 순회하면서 파일의 내용을 읽어서 하나로 만들기
# file은 예약어일 가능성이 높아서 file_로 적음
for file_ in all_files:
    df = pd.read_table(file_, index_col=None, header=0, encoding='latin1')


### 문서 군집

#### 디렉토리 내의 .data로 끝나는 파일을 모두 읽기

In [6]:
import pandas as pd
import glob, os

# 데이터 파일이 있는 디렉토리 경로를 생성
path = "data\\data\\data\\OpinosisDataset1.0\\topics"

# 디렉토리 내의 .data로 끝나는 모든 파일의 이름을 가져오기
all_files = glob.glob(os.path.join(path,"*.data"))
print(all_files)

['data\\data\\data\\OpinosisDataset1.0\\topics\\accuracy_garmin_nuvi_255W_gps.txt.data', 'data\\data\\data\\OpinosisDataset1.0\\topics\\bathroom_bestwestern_hotel_sfo.txt.data', 'data\\data\\data\\OpinosisDataset1.0\\topics\\battery-life_amazon_kindle.txt.data', 'data\\data\\data\\OpinosisDataset1.0\\topics\\battery-life_ipod_nano_8gb.txt.data', 'data\\data\\data\\OpinosisDataset1.0\\topics\\battery-life_netbook_1005ha.txt.data', 'data\\data\\data\\OpinosisDataset1.0\\topics\\buttons_amazon_kindle.txt.data', 'data\\data\\data\\OpinosisDataset1.0\\topics\\comfort_honda_accord_2008.txt.data', 'data\\data\\data\\OpinosisDataset1.0\\topics\\comfort_toyota_camry_2007.txt.data', 'data\\data\\data\\OpinosisDataset1.0\\topics\\directions_garmin_nuvi_255W_gps.txt.data', 'data\\data\\data\\OpinosisDataset1.0\\topics\\display_garmin_nuvi_255W_gps.txt.data', 'data\\data\\data\\OpinosisDataset1.0\\topics\\eyesight-issues_amazon_kindle.txt.data', 'data\\data\\data\\OpinosisDataset1.0\\topics\\featur

In [9]:
# 파일의 이름과 내용을 저장할 list
filename_list = []
opinion_text = []

for file_ in all_files:
    df = pd.read_table(file_, index_col = None, header=0, encoding="latin1")

    # 파일 이름만 추출
    filename_ = file_.split('\\')[-1]
    filename = filename_.split(".")[0]

    # 파일명과 내용을 list에 저장
    filename_list.append(filename)
    opinion_text.append(df.to_string())

print(filename_list)

# 파일 이름과 내용으로 DataFrame을 생성
document_df = pd.DataFrame({'filename':filename_list,'opinion_text':opinion_text})
document_df.head()

['accuracy_garmin_nuvi_255W_gps', 'bathroom_bestwestern_hotel_sfo', 'battery-life_amazon_kindle', 'battery-life_ipod_nano_8gb', 'battery-life_netbook_1005ha', 'buttons_amazon_kindle', 'comfort_honda_accord_2008', 'comfort_toyota_camry_2007', 'directions_garmin_nuvi_255W_gps', 'display_garmin_nuvi_255W_gps', 'eyesight-issues_amazon_kindle', 'features_windows7', 'fonts_amazon_kindle', 'food_holiday_inn_london', 'food_swissotel_chicago', 'free_bestwestern_hotel_sfo', 'gas_mileage_toyota_camry_2007', 'interior_honda_accord_2008', 'interior_toyota_camry_2007', 'keyboard_netbook_1005ha', 'location_bestwestern_hotel_sfo', 'location_holiday_inn_london', 'mileage_honda_accord_2008', 'navigation_amazon_kindle', 'parking_bestwestern_hotel_sfo', 'performance_honda_accord_2008', 'performance_netbook_1005ha', 'price_amazon_kindle', 'price_holiday_inn_london', 'quality_toyota_camry_2007', 'rooms_bestwestern_hotel_sfo', 'rooms_swissotel_chicago', 'room_holiday_inn_london', 'satellite_garmin_nuvi_255W_

,filename,opinion_text
0,accuracy_garmin_nuvi_255W_gps,...
1,bathroom_bestwestern_hotel_sfo,...
2,battery-life_amazon_kindle,...
3,battery-life_ipod_nano_8gb,...
4,battery-life_netbook_1005ha,...


### 피처 벡터화

In [11]:
from nltk.stem import WordNetLemmatizer
import nltk
import string

# 구두점 제거
remove_punch_dict = dict((ord(punch),None) for punch in string.punctuation)
#print(remove_punch_dict)
lemmar = WordNetLemmatizer()

# 문장을 토큰화 한 후 어근을 찾아오는 함수
def LemTokens(tokens):
    return [lemmar.lemmatize(token) for token in tokens]

def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punch_dict)))

from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english',
                               ngram_range=(1,2), min_df=0.05, max_df=0.85)
feature_vect = tfidf_vect.fit_transform(document_df['opinion_text'])

# 문자에 어떤 단어가 얼마의 가중치를 가지고 있는지 희소 행렬로 생성
print(feature_vect)



c:\Users\USER\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
c:\Users\USER\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  warnings.warn(


  (0, 1465)	0.023283541665665836
  (0, 385)	0.023283541665665836
  (0, 1470)	0.05108766228354363
  (0, 2652)	0.023283541665665836
  (0, 1357)	0.02182613746339447
  (0, 723)	0.023283541665665836
  (0, 4367)	0.02182613746339447
  (0, 4593)	0.01798716138533979
  (0, 4273)	0.02063535152550281
  (0, 1468)	0.019628556360506585
  (0, 1804)	0.019628556360506585
  (0, 4040)	0.023283541665665836
  (0, 1467)	0.015101445414252302
  (0, 2469)	0.023283541665665836
  (0, 2173)	0.02182613746339447
  (0, 4191)	0.02182613746339447
  (0, 4196)	0.019628556360506585
  (0, 234)	0.023283541665665836
  (0, 4041)	0.023283541665665836
  (0, 4092)	0.04127070305100562
  (0, 1533)	0.04365227492678894
  (0, 1996)	0.023283541665665836
  (0, 158)	0.02182613746339447
  (0, 1164)	0.023283541665665836
  (0, 3130)	0.02182613746339447
  :	:
  (50, 2028)	0.01659453509406821
  (50, 2037)	0.009111906180203912
  (50, 1655)	0.009761754136944606
  (50, 1321)	0.009452031092583052
  (50, 3294)	0.022126046792090945
  (50, 3430)	0.

#### 군집 알고리즘 수행

In [12]:
from sklearn.cluster import KMeans

# 5개의 군집을 위한 인스턴스 생성
km_cluster = KMeans(n_clusters=5, max_iter=10000, random_state=0)
# 훈련
km_cluster.fit(feature_vect)
# 각 데이터의 레이블을 저장
cluster_label=km_cluster.labels_
# 각 군집의 중앙점을 저장
cluster_centers = km_cluster.cluster_centers_

document_df['cluster_label'] = cluster_label
document_df.head()


c:\Users\USER\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


,filename,opinion_text,cluster_label
0,accuracy_garmin_nuvi_255W_gps,...,2
1,bathroom_bestwestern_hotel_sfo,...,0
2,battery-life_amazon_kindle,...,1
3,battery-life_ipod_nano_8gb,...,1
4,battery-life_netbook_1005ha,...,1


In [13]:
document_df[document_df['cluster_label']==0].sort_values(by='filename')

,filename,opinion_text,cluster_label
1,bathroom_bestwestern_hotel_sfo,...,0
32,room_holiday_inn_london,...,0
30,rooms_bestwestern_hotel_sfo,...,0
31,rooms_swissotel_chicago,...,0


In [14]:
# 군집 개수 변경(5개->3개)
from sklearn.cluster import KMeans

# 3개의 군집을 위한 인스턴스 생성
km_cluster = KMeans(n_clusters=3, max_iter=10000, random_state=0)
# 훈련
km_cluster.fit(feature_vect)
# 각 데이터의 레이블을 저장
cluster_label=km_cluster.labels_
# 각 군집의 중앙점을 저장
cluster_centers = km_cluster.cluster_centers_

document_df['cluster_label'] = cluster_label
document_df.sort_values(by='cluster_label')
document_df.head()


c:\Users\USER\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


,filename,opinion_text,cluster_label
0,accuracy_garmin_nuvi_255W_gps,...,0
1,bathroom_bestwestern_hotel_sfo,...,2
2,battery-life_amazon_kindle,...,0
3,battery-life_ipod_nano_8gb,...,0
4,battery-life_netbook_1005ha,...,0


### 군집을 이루게 만든 핵심 단어 출력

In [16]:
# 클러스터링 모델과 데이터 그리고 피처 이름, 클러스터 개수와 추출할 핵심 단어 개수를 받아서 리턴하는 함수
def get_cluster_details(cluster_model, cluster_data, feature_names, clusters_num,
                        top_n_features=10):
    cluster_details={}

    # 군집 중심과의 거리가 먼 단어 순으로 저장하기
    centroid_feature_ordered_ind = cluster_model.cluster_centers_.argsort()[:,::-1]

    for cluster_num in range(clusters_num):
        # 딕셔터리에다가 하나씩 집어넣기
        cluster_details[cluster_num] = {}
        cluster_details[cluster_num]['cluster'] = cluster_num
        # 중요 피처를 추출
        top_feature_indexes = centroid_feature_ordered_ind[cluster_num, :top_n_features]
        top_features = [feature_names[ind] for ind in top_feature_indexes]
        # 중앙점과의 거리 저장
        top_feature_values = cluster_model.cluster_centers_[cluster_num, top_feature_indexes].tolist()

        cluster_details[cluster_num]['top_features'] = top_features
        cluster_details[cluster_num]['top_features_value'] = top_feature_values
        filenames = cluster_data[cluster_data['cluster_label'] == cluster_num]['filename']
        filenames = filenames.values.tolist()
        cluster_details[cluster_num]['filenames'] = filenames

    return cluster_details

In [17]:
# 클러스터별 핵심 단어를 출력하는 함수

def print_cluster_details(cluster_details):
    for cluster_num, cluster_detail in cluster_details.items():
        print('#######', cluster_num, '#######')
        print('핵심 단어:', cluster_detail['top_features'])
        print('파일명:',cluster_detail['filenames'])


In [20]:
# 피처 이름을 전부 가져오기
feature_names = tfidf_vect.get_feature_names_out()
#print(feature_names)
cluster_details = get_cluster_details(cluster_model=km_cluster, cluster_data=document_df,
                                  feature_names=feature_names, clusters_num=3, top_n_features=10)
print_cluster_details(cluster_details)


####### 0 #######
핵심 단어: ['screen', 'battery', 'keyboard', 'battery life', 'life', 'kindle', 'direction', 'video', 'size', 'voice']
파일명: ['accuracy_garmin_nuvi_255W_gps', 'battery-life_amazon_kindle', 'battery-life_ipod_nano_8gb', 'battery-life_netbook_1005ha', 'buttons_amazon_kindle', 'directions_garmin_nuvi_255W_gps', 'display_garmin_nuvi_255W_gps', 'eyesight-issues_amazon_kindle', 'features_windows7', 'fonts_amazon_kindle', 'keyboard_netbook_1005ha', 'navigation_amazon_kindle', 'performance_netbook_1005ha', 'price_amazon_kindle', 'satellite_garmin_nuvi_255W_gps', 'screen_garmin_nuvi_255W_gps', 'screen_ipod_nano_8gb', 'screen_netbook_1005ha', 'size_asus_netbook_1005ha', 'sound_ipod_nano_8gb', 'speed_garmin_nuvi_255W_gps', 'speed_windows7', 'updates_garmin_nuvi_255W_gps', 'video_ipod_nano_8gb', 'voice_garmin_nuvi_255W_gps']
####### 1 #######
핵심 단어: ['interior', 'seat', 'mileage', 'comfortable', 'gas', 'gas mileage', 'transmission', 'car', 'performance', 'quality']
파일명: ['comfort_honda

#### 코사인 유사도를 구하는 알고리즘

In [21]:
# 코사인 유사도를 계산해주는 함수
def cos_similarity(v1, v2) :
    dot_product = np.dot(v1,v2)
    l2_norm = (np.sqrt(sum(np.square(v1))) * np.sqrt(sum(np.square(v2))))
    similarity = dot_product / l2_norm

    return similarity

In [28]:
doc_list = ['if you take the blue pill, the story ends',
            'if you take the red pill, you stay in Wonderland',
            'if you take the red pill, i show you how deep rabbit hole goes']

tfidf_vect_simple = TfidfVectorizer()
feature_vect_simple = tfidf_vect_simple.fit_transform(doc_list)

# 현재 결과는 희소 행렬이라서 거리 계산을 못함
# 단어의 개수가 달라서 계산 불가(각각의 피처 개수가 다름)
print(feature_vect_simple[0])
print("==")
print(feature_vect_simple[1])

# 밀집 배열로 변환(피처의 개수가 같아졌음 : 거리 계산 가능)
feature_vect_dense = feature_vect_simple.todense()
print("==")
print(feature_vect_dense[0])
print("==")
print(feature_vect_dense[1])


  (0, 2)	0.41556360057939173
  (0, 13)	0.41556360057939173
  (0, 8)	0.24543855687841593
  (0, 0)	0.41556360057939173
  (0, 15)	0.49087711375683185
  (0, 14)	0.24543855687841593
  (0, 17)	0.24543855687841593
  (0, 6)	0.24543855687841593
==
  (0, 16)	0.39624495215024286
  (0, 7)	0.39624495215024286
  (0, 12)	0.39624495215024286
  (0, 10)	0.3013544995034864
  (0, 8)	0.2340286519091622
  (0, 15)	0.2340286519091622
  (0, 14)	0.2340286519091622
  (0, 17)	0.4680573038183244
  (0, 6)	0.2340286519091622
==
[[0.4155636  0.         0.4155636  0.         0.         0.
  0.24543856 0.         0.24543856 0.         0.         0.
  0.         0.4155636  0.24543856 0.49087711 0.         0.24543856]]
==
[[0.         0.         0.         0.         0.         0.
  0.23402865 0.39624495 0.23402865 0.         0.3013545  0.
  0.39624495 0.         0.23402865 0.23402865 0.39624495 0.4680573 ]]


In [30]:
# 거리 계산
import numpy as np

vect1 = np.array(feature_vect_dense[0]).reshape(-1,)
vect2 = np.array(feature_vect_dense[1]).reshape(-1,)
vect3 = np.array(feature_vect_dense[2]).reshape(-1,)

print("문장 1과 문장2의 유사도:", cos_similarity(vect1, vect2))
print("문장 1과 문장3의 유사도:", cos_similarity(vect1, vect3))
print("문장 2과 문장3의 유사도:", cos_similarity(vect2, vect3))

문장 1과 문장2의 유사도: 0.4020775821495014
문장 1과 문장3의 유사도: 0.33151185783991466
문장 2과 문장3의 유사도: 0.4361341528640473


### API를 활용한 코사인 유사도 구하기

In [31]:
from sklearn.metrics.pairwise import cosine_similarity
similarity_simple_pair = cosine_similarity(feature_vect_simple[0],feature_vect_simple)

print(similarity_simple_pair)

[[1.         0.40207758 0.33151186]]


### 문서 군집의 코사인 유사도 확인

In [32]:
km_cluster = KMeans(n_clusters=3, max_iter=10000, random_state=0)
km_cluster.fit(feature_vect)
cluster_label = km_cluster.labels_
# 각 군집의 중앙점(centroid)
cluster_centers = km_cluster.cluster_centers_
document_df['cluster_label'] = cluster_label
print(cluster_centers)

c:\Users\USER\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


[[0.01005322 0.         0.         ... 0.00706287 0.         0.        ]
 [0.         0.00092551 0.         ... 0.         0.         0.        ]
 [0.         0.00099499 0.00174637 ... 0.         0.00183397 0.00144581]]


In [37]:
# 1번 클러스터의 문서들 간의 코사인 유사도 확인
hotel_indexes = document_df[document_df['cluster_label']==1].index
print("1번 클러스터의 문서 인덱스:", hotel_indexes)


# 자신의 군집에 있는 문서와의 코사인 유사도
similarity_pair = cosine_similarity(feature_vect[hotel_indexes[0]],
                                    feature_vect[hotel_indexes])
print(similarity_pair)

# 0번 부터 7번 까지 문서와 코사인 유사도(6,7번째만 1번 클러스트이고 나머지는 아님)
similarity_pair = cosine_similarity(feature_vect[hotel_indexes[0]],
                                    feature_vect[[0,1,2,3,4,5,6,7]])
print(similarity_pair)

1번 클러스터의 문서 인덱스: Int64Index([6, 7, 16, 17, 18, 22, 25, 29, 37, 47], dtype='int64')
[[1.         0.83969704 0.15655631 0.33044002 0.25981841 0.16544257
  0.27569738 0.18050974 0.65502034 0.06229873]]
[[0.02315315 0.03787848 0.01383035 0.01725198 0.01973879 0.01873283
  1.         0.83969704]]


### 한글 문서의 유사도 측정

In [41]:
from sklearn.feature_extraction.text import CountVectorizer
# Twitter 에서 Okt로 바꼈다는 경고가 떠서 Okt로 바꿔줌
from konlpy.tag import Okt

okt = Okt()
vectorizer = CountVectorizer(min_df=1)
contents = ['우리 과일 먹으러 가자', '오늘은 목요일입니다.',
            '나는 공원에서 산책하는 것을 싫어합니다.',
            '나는 거리를 걷는 것을 좋아합니다.',
            '걷는 것이 프로그래밍보다 재미있습니다.']

# 한글 토큰화
contents_tokens = [okt.morphs(row) for row in contents]
print(contents_tokens)


[['우리', '과일', '먹으러', '가자'], ['오늘', '은', '목요일', '입니다', '.'], ['나', '는', '공원', '에서', '산책', '하는', '것', '을', '싫어합니다', '.'], ['나', '는', '거리', '를', '걷는', '것', '을', '좋아합니다', '.'], ['걷는', '것', '이', '프로그래밍', '보다', '재미있습니다', '.']]


In [42]:
# 토큰화된 결과를 가지고 다시 문장을 생성 - 피처 벡터화 때문
contents_for_vectorize=[]
for content in contents_tokens:
    sentence = ''
    for word in content:
        sentence = sentence + ' ' + word
    contents_for_vectorize.append(sentence)
print(contents_for_vectorize)

[' 우리 과일 먹으러 가자', ' 오늘 은 목요일 입니다 .', ' 나 는 공원 에서 산책 하는 것 을 싫어합니다 .', ' 나 는 거리 를 걷는 것 을 좋아합니다 .', ' 걷는 것 이 프로그래밍 보다 재미있습니다 .']


In [43]:
# 피처 벡터화
X = vectorizer.fit_transform(contents_for_vectorize)

# 피처 확인
print(vectorizer.get_feature_names_out())

# 문장의 피처 벡터화 된 후의 결과 확인
print(X.toarray().transpose())



['가자' '거리' '걷는' '공원' '과일' '먹으러' '목요일' '보다' '산책' '싫어합니다' '에서' '오늘' '우리'
 '입니다' '재미있습니다' '좋아합니다' '프로그래밍' '하는']
[[1 0 0 0 0]
 [0 0 0 1 0]
 [0 0 0 1 1]
 [0 0 1 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [0 1 0 0 0]
 [0 0 0 0 1]
 [0 0 1 0 0]
 [0 0 1 0 0]
 [0 0 1 0 0]
 [0 1 0 0 0]
 [1 0 0 0 0]
 [0 1 0 0 0]
 [0 0 0 0 1]
 [0 0 0 1 0]
 [0 0 0 0 1]
 [0 0 1 0 0]]


In [44]:
# 유사도를 측정할 데이터 생성
# 토큰화 후 다시 문장 생성
new_post = ['우리 과이 먹으러 갖']
new_post_tokens = [okt.morphs(row) for row in new_post]

new_post_for_vectorize = []

for content in new_post_tokens:
    sentence = ''
    for word in content:
        sentence = sentence + ' ' + word
        
    new_post_for_vectorize.append(sentence)
    
print(new_post_for_vectorize)


[' 우리 과 이 먹으러 갖']


In [46]:
# 테스트 데이터의 피처 벡터화
new_post_vec = vectorizer.transform(new_post_for_vectorize)
print(new_post_vec)
print(new_post_vec.toarray())

  (0, 5)	1
  (0, 12)	1
[[0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0]]


In [47]:
# 거리 구하는 함수 만들기
import scipy as sp

def dist_raw(v1, v2):
    delta = v1 - v2
    return sp.linalg.norm(delta.toarray())

best_doc = None
best_dist = 65535
best_i = None

In [48]:
# 다른 문장들과의 거리
best_doc = None
best_dist = 65535
best_i = None

for i in range(0,5):
    post_vec = X.getrow(i)
    d = dist_raw(post_vec, new_post_vec)

    print(i, "번째 문장과의 거리:", d, " ", contents[i])

    if d < best_dist:
        best_dist = d
        best_i = i



0 번째 문장과의 거리: 1.4142135623730951   우리 과일 먹으러 가자
1 번째 문장과의 거리: 2.23606797749979   오늘은 목요일입니다.
2 번째 문장과의 거리: 2.6457513110645907   나는 공원에서 산책하는 것을 싫어합니다.
3 번째 문장과의 거리: 2.23606797749979   나는 거리를 걷는 것을 좋아합니다.
4 번째 문장과의 거리: 2.449489742783178   걷는 것이 프로그래밍보다 재미있습니다.


In [49]:
print(contents[best_i])

우리 과일 먹으러 가자
